# Graphical Database
A graphical database, such as Neo4J, offers a number of advantages when compared to traditional relational databases. One of the main benefits of using a graphical database is the ability to easily visualize and explore the relationships between different data entities. In the case of our artworks dataset, this means that we will be able to see the connections between artists, artworks, movements, mediums, and other related concepts in a way that is easy to understand and interact with. Additionally, the use of a graph database will allow us to easily search for and retrieve information based on the relationships between entities. This can be particularly useful when exploring complex connections and relationships within the data. In this notebook, we will demonstrate how to upload the data described above to a Neo4J server and transform our relational database into a graph database. This will provide us with a new way of exploring and understanding our data, and will help to uncover new insights and relationships that may have been previously hidden.

Neo4j is a popular graph database management system that is used to store and manage complex, connected data. It allows you to represent your data as nodes and relationships between nodes, making it a powerful tool for exploring and querying large datasets. The data is stored in a graph structure, making it quick and easy to traverse and search the data. If you would like to learn more about Neo4j, you can visit their website at neo4j.com. To download Neo4j, simply go to the download section of their website and select the version that best fits your needs. After downloading, follow the instructions to install and set up a local instance of Neo4j on your machine.

The course website of Data Management [B-KUL-G0Z53A] can also be a useful resource: https://neo4j-website.s3.eu-central-1.amazonaws.com/build/html/GraphDatabases/index.html.

In [4]:
# PARAMETERS
YOUR_PASSWORD: str = "password"
YOUR_PORT: int = 7687 # This should probably not be changed

**INSTRUCTIONS**

1. Realize that uploading the data to Neo4J is optional
2. Create and start a local database instance

![Local Database Instance](https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/database_instance.png)

3. Modify the parameters above to match your specifications
4. Run all cells and pray
5. Open your Neo4J browser and check if the data is there
6. Use the command 'call db.schema.visualization'

![Database schema](https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/database_schema.png)

In [5]:
import pandas as pd
from py2neo import Graph

graph = Graph(f"bolt://localhost:{YOUR_PORT}",password=YOUR_PASSWORD)
graph.run('match (n) detach delete n') # Drops all data
try:
    indexes = graph.run('show indexes yield name').to_data_frame()['name'] # drops all indices
    for index in indexes:
        graph.run(f'drop index {index}')
except:
    pass

ConnectionUnavailable: Cannot open connection to ConnectionProfile('bolt://localhost:7687')

## Specialization

In [3]:
specialization = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Specialization.parquet.gzip')
specialization.head(2)

,id,name,description
0,0,Still Life,A still life (plural: still lifes) is a work o...
1,1,Portraits,"A portrait is a painting, photograph, sculptur..."


In [4]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Specialization.csv" AS csvLine
MERGE (s:Specialization {
    id: toInteger(csvLine.id), 
    name: csvLine.name,
    description: csvLine.description
    })
"""
)

(No data)

In [5]:
graph.run('CREATE INDEX specialization FOR (n:Specialization) ON (n.id)')

(No data)

### Movement

In [6]:
movement = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Movement.parquet.gzip')
movement.head(2)

,id,name,description
0,0,Ashcan School,"The Ashcan School, also called the Ash Can Sch..."
1,1,Pre-Raphaelite Brotherhood,The Pre-Raphaelite Brotherhood (later known as...


In [7]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Movement.csv" AS csvLine
MERGE (m:Movement {
    id: toInteger(csvLine.id), 
    name: csvLine.name,
    description: csvLine.description
    })
"""
)

(No data)

In [8]:
graph.run('CREATE INDEX movement FOR (n:Movement) ON (n.id)')

(No data)

### Academy

In [9]:
academy = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Academy.parquet.gzip')
academy.head(2)

,id,name,description
0,0,Royal Academy of Fine Arts Antwerp,The Royal Academy of Fine Arts Antwerp (Dutch:...
1,1,Académie Royale des Beaux-Arts,The Royal Academy of Fine Arts of Brussels (Fr...


In [10]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Academy.csv" AS csvLine
MERGE (a:Academy {
    id: toInteger(csvLine.id), 
    name: csvLine.name,
    description: csvLine.description
    })
"""
)

(No data)

In [11]:
graph.run('CREATE INDEX academy FOR (n:Academy) ON (n.id)')

(No data)

### Medium

In [12]:
medium = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Medium.parquet.gzip')
medium.head(2)

,id,name,description
0,0,Oil on canvas,Oil painting is the process of painting with p...
1,1,Oil on panel,Oil painting is the process of painting with p...


In [13]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Medium.csv" AS csvLine
MERGE (m:Medium {
    id: toInteger(csvLine.id), 
    name: csvLine.name,
    description: csvLine.description
    })
"""
)

(No data)

In [14]:
graph.run('CREATE INDEX medium FOR (n:Medium) ON (n.id)')

(No data)

### Places

In [15]:
places = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Places.parquet.gzip')
places.head(2)

,id,name,parent
0,0,Zundert,213.0
1,1,Limoges,214.0


In [16]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Places.csv" AS csvLine
MERGE (m:Place {
    id: toInteger(csvLine.id), 
    name: csvLine.name
    })
"""
)

(No data)

In [17]:
graph.run('CREATE INDEX place FOR (n:Place) ON (n.id)')

(No data)

##### Relations

In [18]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Places.csv" AS csvLine
MATCH (p1:Place {id: toInteger(csvLine.id)}), (p2:Place {id: toInteger(csvLine.parent)})
MERGE (p1) -[r:LOCATED_IN]-> (p2)
"""
)

(No data)

### Artist Pictures

In [19]:
pictures = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/ArtistPicture.parquet.gzip')
pictures.head(2)

,id,url,source_url,caption
0,0,https://kuleuven-datathon-2023.s3.eu-central-1...,http://upload.wikimedia.org/wikipedia/commons/...,"Self-Portrait, 1887, Art Institute of Chicago"
1,1,https://kuleuven-datathon-2023.s3.eu-central-1...,http://upload.wikimedia.org/wikipedia/commons/...,None


In [20]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/ArtistPicture.csv" AS csvLine
CREATE (m:Picture {
    id: toInteger(csvLine.id), 
    url: csvLine.url,
    source_url: csvLine.source_url,
    caption: csvLine.caption
    })
"""
)

(No data)

In [21]:
graph.run('CREATE INDEX picture FOR (n:Picture) ON (n.id)')

(No data)

### Generated artworks

In [39]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Generated.csv" AS csvLine
CREATE (m:Generated {
    url: csvLine.url
    })
"""
)

(No data)

### Artwork

In [22]:
artworks = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artwork.parquet.gzip')
artworks.head(2)

,id,name,url,image_url,artist,rating,summary,year,medium,location
0,0,Cafe Terrace on the Place du Forum,http://wikigallery.org/wiki/painting_133032/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,Café Terrace at Night is an 1888 oil painting ...,1888.0,0.0,Kröller-Müller Museum
1,1,Starry Night,http://wikigallery.org/wiki/painting_9698/Vinc...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,None,NaN,NaN,None


In [23]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artwork.csv" AS csvLine
CREATE (m:Artwork {
    id: toInteger(csvLine.id), 
    name: csvLine.name,
    image_url: csvLine.image_url,
    rating: toInteger(csvLine.rating),
    summary: csvLine.summary,
    year: toIntegerOrNull(csvLine.year),
    location: csvLine.location
    })
"""
)

(No data)

In [24]:
graph.run('CREATE INDEX artwork FOR (n:Artwork) ON (n.id)')

(No data)

#### Relation to medium

In [25]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artwork.csv" AS csvLine
MATCH (a:Artwork {id: toInteger(csvLine.id)}), (m:Medium {id: toIntegerOrNull(csvLine.medium)})
MERGE (a) -[r:USES]-> (m)
"""
)

(No data)

#### Relation with generated

In [40]:
generated = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Generated.parquet.gzip')
generated.head(2)

,source_artwork,url
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1...
1,1480,https://kuleuven-datathon-2023.s3.eu-central-1...


In [41]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Generated.csv" AS csvLine
MATCH (a:Artwork {id: toInteger(csvLine.source_artwork)}), (g:Generated {url: csvLine.url})
MERGE (g) -[r:BASED_ON]-> (a)
"""
)

(No data)

#### Recommendations

In [26]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Recommendation.csv" AS csvLine
MATCH (a:Artwork {id: toInteger(csvLine.artwork)}), (recommendation:Artwork {id: toInteger(csvLine.recommended)})
MERGE (a) -[r:RECOMMENDS]-> (recommendation)
"""
)

(No data)

### Artists

In [27]:
artists = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.parquet.gzip')
artists.fillna(pd.NA, inplace=True)
artists.head(2)

,id,name,url,summary,picture,birthplace,deathplace,birthdate,deathdate,cause_of_death
0,0,Vincent Van Gogh,http://wikigallery.org/wiki/artist36933/Vincen...,Vincent Willem van Gogh (Dutch: [ˈvɪnsɛnt ˈʋɪl...,0.0,0.0,342.0,1853-03-30,1890-07-29,Gunshot wound
1,1,Pierre Auguste Renoir,http://wikigallery.org/wiki/artist39254/Pierre...,Pierre-Auguste Renoir (French: [pjɛʁ oɡyst ʁən...,1.0,1.0,343.0,1841-02-25,1919-12-03,<NA>


In [28]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.csv" AS csvLine
CREATE (m:Artist {
    id: toInteger(csvLine.id), 
    name: csvLine.name,
    url: csvLine.url,
    summary: csvLine.summary
    })
"""
)

(No data)

In [29]:
graph.run('CREATE INDEX artist FOR (n:Artist) ON (n.id)')

(No data)

#### Relationship to picture

In [30]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.csv" AS csvLine
MATCH (a:Artist {id: toInteger(csvLine.id)}), (picture:Picture {id: toIntegerOrNull(csvLine.picture)})
MERGE (a) -[r:IMAGE]-> (picture)
"""
)

(No data)

##### Relationship to birth and deathplace

In [31]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.csv" AS csvLine
MATCH (a:Artist {id: toInteger(csvLine.id)}), (birthplace:Place {id: toIntegerOrNull(csvLine.birthplace)})
MERGE (a) -[r:BORN_IN]-> (birthplace)
"""
)

(No data)

In [32]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.csv" AS csvLine
MATCH (a:Artist {id: toInteger(csvLine.id)}), (deathplace:Place {id: toIntegerOrNull(csvLine.deathplace)})
MERGE (a) -[r:DIED_IN]-> (deathplace)
SET r.cause = csvLine.cause_of_death
"""
)

(No data)

##### Apprenticeships

In [33]:
pd.read_csv('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Apprenticeship.csv').head()

,student_id,teacher_id
0,4.0,301.0
1,4.0,577.0
2,6.0,578.0
3,15.0,579.0
4,15.0,580.0


In [34]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Apprenticeship.csv" AS csvLine
MATCH (student:Artist {id: toInteger(csvLine.student_id)}), (teacher:Artist {id: toIntegerOrNull(csvLine.teacher_id)})
MERGE (student) -[r:APPRENTICE_OF]-> (teacher)
"""
)

(No data)

### Relations

##### Artist - Artwork

In [35]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artwork.csv" AS csvLine
MATCH (artwork:Artwork {id: toInteger(csvLine.id)}), (artist:Artist {id: toIntegerOrNull(csvLine.artist)})
MERGE (artwork) -[r:MADE_BY]-> (artist)
"""
)

(No data)

#### Artist - Specialization/Movement/Education

In [36]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/ArtistSpecializations.csv" AS csvLine
MATCH (s:Specialization {id: toInteger(csvLine.specialty_id)}), (artist:Artist {id: toIntegerOrNull(csvLine.artist_id)})
MERGE (s) <-[r:SPECIALIZED_IN]- (artist)
"""
)

(No data)

In [37]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/ArtistMovements.csv" AS csvLine
MATCH (s:Movement {id: toInteger(csvLine.movement_id)}), (artist:Artist {id: toIntegerOrNull(csvLine.artist_id)})
MERGE (s) <-[r:BELONGS_TO]- (artist)
"""
)

(No data)

In [38]:
graph.run(
"""
LOAD CSV WITH HEADERS FROM "https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/ArtistEducation.csv" AS csvLine
MATCH (s:Academy {id: toInteger(csvLine.academy_id)}), (artist:Artist {id: toIntegerOrNull(csvLine.artist_id)})
MERGE (s) <-[r:EDUCATED_AT]- (artist)
"""
)

(No data)